In [165]:
import re
import camelot
import os
import csv
import pdfplumber
import re

In [166]:
file_path = "./octa+-dynamic-2023-05.pdf"

In [167]:
csv_file = "octaplus_dynamic.csv"

In [168]:
contract_key = "OCTAPLUS_DYNAMIC"

In [169]:
data = dict()

In [170]:
tables = camelot.read_pdf(file_path, pages="all", flavor="stream")

Overwriting cache for 0 389
Overwriting cache for 0 389
C:\Users\jaakd\PycharmProjects\dep1-24-25-g02\venv\Lib\site-packages\camelot\parsers\base.py:238: UserWarning: No tables found in table area (233.47999999999996, 573.24, 361.71616400000005, 712.3271597047658)
  cols, rows, v_s, h_s = self._generate_columns_and_rows(bbox, user_cols)
Overwriting cache for 0 389
Overwriting cache for 0 389
C:\Users\jaakd\PycharmProjects\dep1-24-25-g02\venv\Lib\site-packages\camelot\parsers\base.py:238: UserWarning: No tables found in table area (35.59999999999998, 387.96999999999997, 561.0328000000002, 853.3999999999997)
  cols, rows, v_s, h_s = self._generate_columns_and_rows(bbox, user_cols)


In [171]:
for i, table in enumerate(tables):
    df = table.df
    if df.isin(["Vaste vergoeding (€/jaar)"]).any().any():
        fixed_fee_row = df.index[df.isin(["Vaste vergoeding (€/jaar)"]).any(axis=1)][0]
        number = re.findall(r'-?\d+,\d+', df.loc[fixed_fee_row][1])[0].replace(",", '.')
        data['AdministrativeCosts'] = float(number)

In [172]:
flatten = lambda *n: (e for a in n for e in (flatten(*a) if isinstance(a, (tuple, list)) else (a,)))

In [173]:
with pdfplumber.open(file_path) as pdf:
    tables = []
    for page in pdf.pages:
        tables.append(tables.append(page.extract_tables()))
    cells = list(flatten(tables))

In [174]:
for cell in cells:
    if "Kosten WKK" in str(cell):
        numbers = re.findall(r'-?\d+,\d+', cell)
        numbers = [float(num.replace(',', '.')) for num in numbers]
        data['GreenElectricity'], data["WKK"] = numbers
        break

In [175]:
with pdfplumber.open(file_path) as pdf:
    for page in pdf.pages:
        if "De elektriciteitsprijs voor OCTA+ Dynamic wordt elk uur geïndexeerd" in page.extract_text():
            for line in page.extract_text().split("\n"):
                if "als volgt: Belpex Hourly *" in line:
                    numbers = re.findall(r'-?\d+,\d+', line)
                    numbers = [float(num.replace(',', '.')) for num in numbers]
                    data['DynamicMeterCost'], data["DynamicBalancingCost"] = numbers
                    break

In [176]:
data

{'AdministrativeCosts': 75.0,
 'GreenElectricity': 2.279,
 'WKK': 0.344,
 'DynamicMeterCost': 1.038,
 'DynamicBalancingCost': 3.93}

In [177]:
year, month = re.search(r'(\d{4})-(\d{2})', file_path).groups()
date_key = f"{year}{month}01"  # Format as yyyymmdd with day as 01

In [178]:
file_exists = os.path.isfile(csv_file)

In [179]:
with open(csv_file, mode='a', newline='', encoding='utf-8') as file:
    writer = csv.writer(file, delimiter=';')
    
    # Write header only if file doesn't exist
    if not file_exists:
        header = ["DateKey", "ContractKey", "SingleMeterFixed", "DualMeterDayFixed", 
                  "DualMeterNightFixed", "ExclusiveNightMeterFixed", "SingleMeterVariableMeterFactor", 
                  "SingleMeterVariableBalancingCost", "DualMeterDayVariableMeterFactor", 
                  "DualMeterDayVariableBalancingCost", "DualMeterNightVariableMeterFactor", 
                  "DualMeterNightVariableBalancingCost", "ExclusiveNightMeterVariableMeterFactor", 
                  "ExclusiveNightMeterVariableBalancingCost", "DynamicMeterCost", "DynamicBalancingCost", 
                  "SingleMeterInjectionMeterFactor", "SingleMeterInjectionBalancingCost", 
                  "DualMeterDayInjectionMeterFactor", "DualMeterDayInjectionBalancingCost", 
                  "DualMeterNightInjectionMeterFactor", "DualMeterNightInjectionBalancingCost", 
                  "AdministrativeCosts", "GreenElectricity", "WKK"]
        writer.writerow(header)
    
    # Prepare data row in the specified order
    row_data = [
        date_key,
        contract_key,
        data.get('SingleMeterFixed', ''),
        data.get('DualMeterDayFixed', ''),
        data.get('DualMeterNightFixed', ''),
        data.get('ExclusiveNightMeterFixed', ''),
        data.get('SingleMeterVariableMeterFactor', ''),
        data.get('SingleMeterVariableBalancingCost', ''),
        data.get('DualMeterDayVariableMeterFactor', ''),
        data.get('DualMeterDayVariableBalancingCost', ''),
        data.get('DualMeterNightVariableMeterFactor', ''),
        data.get('DualMeterNightVariableBalancingCost', ''),
        data.get('ExclusiveNightMeterVariableMeterFactor', ''),
        data.get('ExclusiveNightMeterVariableBalancingCost', ''),
        data.get('DynamicMeterCost', ''),
        data.get('DynamicBalancingCost', ''),
        data.get('SingleMeterInjectionMeterFactor', ''),
        data.get('SingleMeterInjectionBalancingCost', ''),
        data.get('DualMeterDayInjectionMeterFactor', ''),
        data.get('DualMeterDayInjectionBalancingCost', ''),
        data.get('DualMeterNightInjectionMeterFactor', ''),
        data.get('DualMeterNightInjectionBalancingCost', ''),
        data.get('AdministrativeCosts', ''),
        data.get('GreenElectricity', ''),
        data.get('WKK', '')
    ]

    
    # Write data row
    writer.writerow(row_data)

print(f"Data for {date_key} successfully written to {csv_file}")

Data for 20230501 successfully written to octaplus_dynamic.csv
